In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=271b8feb43272308794f576f1e8ac09288250c6b147cdc2bfd54b694bfa5235b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *


In [4]:
spark = SparkSession.builder.appName("Read lines from a file stream").getOrCreate()

In [5]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [42]:
schema = StructType([StructField('ID', IntegerType(), True),
                       StructField('Date', DateType(), True),
                       StructField('Open', FloatType(), True),
                       StructField('High', FloatType(), True),
                       StructField('Low', FloatType(), True),
                       StructField('Close', FloatType(), True),
                       StructField('Adj Close', FloatType(), True),
                       StructField('Volume', IntegerType(), True)])



### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [67]:
  df = spark.readStream.format("csv").schema(schema).load("/content/drive/MyDrive/pyspark/kospi")


### Make sure the dataframe is streaming the files from the folder

In [68]:
df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:


In [69]:
writer = df.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('stock') \
    .trigger(processingTime='4 seconds')

### Start the write stream and make sure it works (read all columns from the table)

In [70]:
query= writer.start()

In [71]:
df_st = spark.sql('SELECT * FROM stock').show()

+----+----------+-------+-------+-------+-------+---------+------+
|  ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|
+----+----------+-------+-------+-------+-------+---------+------+
|NULL|      NULL|   NULL|   NULL|   NULL|   NULL|     NULL|  NULL|
| 240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019|
| 241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791|
| 242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|
| 243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|
| 244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|
| 245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560|
| 246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|
| 247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|
| 248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|
| 249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|
| 250|2000-12-19|27722.7|28788.9|27651.6|27651.6|  26130.7|115

In [72]:
df_st = spark.sql('SELECT * FROM stock').show(100)

+----+----------+-------+-------+-------+-------+---------+------+
|  ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|
+----+----------+-------+-------+-------+-------+---------+------+
|NULL|      NULL|   NULL|   NULL|   NULL|   NULL|     NULL|  NULL|
| 240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019|
| 241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791|
| 242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|
| 243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|
| 244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|
| 245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560|
| 246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|
| 247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|
| 248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|
| 249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|
| 250|2000-12-19|27722.7|28788.9|27651.6|27651.6|  26130.7|115

+----+----------+------------+------------+------------+------------+------------+------+
|  ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+----+----------+------------+------------+------------+------------+------------+------+
|null|      null|        null|        null|        null|        null|        null|  null|
| 120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|
| 121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651|
| 122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209|
| 123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|
| 124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|
| 125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|
| 126|2000-06-28|23884.199219|24666.099609|23884.199219|24666.099609|23309.408203| 86236|
| 127|2000

In [100]:
query.stop()

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [73]:
df_dropna = df.na.drop(how='all')
df_diff = df_dropna.withColumn("diff", df.High - df.Low)
df_diff.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- diff: float (nullable = true)



### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [83]:
writer_mod = df_diff.writeStream.format("memory") \
    .queryName("modified_data") \
    .trigger(processingTime='4 seconds') \
    .outputMode("append")


In [84]:
query_mod = writer_mod.start()

In [87]:
df_st_mod = spark.sql('SELECT * FROM modified_data').show()

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019| 995.2012|
|241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791| 710.8008|
|242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|533.10156|
|243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|1066.2988|
|244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|1208.4004|
|245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560| 639.7988|
|246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|1634.9004|
|247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|1492.7988|
|248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|1101.7988|
|249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|1066.1992|
|250|2000-12

+---+----+----+----+---+-----+---------+------+----+
| ID|Date|Open|High|Low|Close|Adj Close|Volume|diff|
+---+----+----+----+---+-----+---------+------+----+
+---+----+----+----+---+-----+---------+------+----+



In [88]:
df_st_mod = spark.sql('SELECT * FROM modified_data').show(150)

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019| 995.2012|
|241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791| 710.8008|
|242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|533.10156|
|243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|1066.2988|
|244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|1208.4004|
|245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560| 639.7988|
|246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|1634.9004|
|247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|1492.7988|
|248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|1101.7988|
|249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|1066.1992|
|250|2000-12

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|1421.5996099999975|
|121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651| 995.0996099999975|
|122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209| 924.0996090000008|
|123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|1990.3007819999984|
|124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|1599.4003909999992|
|125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|1777.0996090000008|
|126|2000-06-28|23884.199219

In [99]:
query_mod.stop()

### Write the generated data into files instead of the memory.

In [101]:
writer_file = df_diff.writeStream.outputMode("append") \
    .format("csv")  \
    .option("path", "outputstream") \
    .option("checkpointLocation", "chkpoint") \
    .trigger(processingTime='4 seconds')

In [102]:
query_file = writer_file.start()

In [107]:
query_file.stop()

### Stop the query. Now, try reading the generated files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [104]:
schema2 = StructType([StructField('ID', IntegerType(), True),
                       StructField('Date', DateType(), True),
                       StructField('Open', FloatType(), True),
                       StructField('High', FloatType(), True),
                       StructField('Low', FloatType(), True),
                       StructField('Close', FloatType(), True),
                       StructField('Adj Close', FloatType(), True),
                       StructField('Volume', IntegerType(), True),
                       StructField('diff',FloatType(),True)])


In [115]:
  df2 = spark.read.format("csv").schema(schema2).load("/content/outputstream")


In [117]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Adj Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- diff: float (nullable = true)



In [118]:
df2.show(150)

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|240|2000-12-05|26585.3|27367.3|26372.1|27011.8|25526.092| 91019| 995.2012|
|241|2000-12-06|27011.8|27509.4|26798.6|26869.7|25391.805|105791| 710.8008|
|242|2000-12-07|27011.8|27011.8|26478.7|26656.4|25190.236| 40656|533.10156|
|243|2000-12-08|26656.4|27722.7|26656.4|27651.6|  26130.7|149964|1066.2988|
|244|2000-12-11|27687.1|28860.0|27651.6|28078.1| 26533.74|159671|1208.4004|
|245|2000-12-12|28042.6|28078.1|27438.3|27935.9|26399.361| 74560| 639.7988|
|246|2000-12-13|27651.6|29286.5|27651.6|28469.1|26903.234|270385|1634.9004|
|247|2000-12-14|28469.1|29784.1|28291.3|28362.4|26802.406|256317|1492.7988|
|248|2000-12-15|28362.4|28895.6|27793.8|27935.9|26399.361|108886|1101.7988|
|249|2000-12-18|27580.5|28433.5|27367.3|28291.3|26735.217| 92848|1066.1992|
|250|2000-12

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|1421.5996099999975|
|121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651| 995.0996099999975|
|122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209| 924.0996090000008|
|123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|1990.3007819999984|
|124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|1599.4003909999992|
|125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|1777.0996090000008|
|126|2000-06-28|23884.199219

### Sort the dataframe based on the ID

In [119]:
finalDFSorted = df2.sort(df2.ID)
finalDFSorted.show()

+---+----------+-------+-------+-------+-------+---------+------+---------+
| ID|      Date|   Open|   High|    Low|  Close|Adj Close|Volume|     diff|
+---+----------+-------+-------+-------+-------+---------+------+---------+
|  0|2000-01-04|22817.9|25696.8|22817.9|24879.3| 23510.88|108745|2878.9004|
|  1|2000-01-05|24523.9|26229.9|23670.9|24417.3|23074.295|175990|   2559.0|
|  2|2000-01-06|24381.7|24666.1|22746.8|22817.9|21562.865| 71746|1919.2988|
|  3|2000-01-07|22036.0|24879.3|22036.0|23884.2|22570.514|120984|2843.3008|
|  4|2000-01-10|24879.3|25519.1|23813.1|24061.9| 22738.44|151371|   1706.0|
|  5|2000-01-11|24168.5|25021.5|23955.2|24239.6|22906.365| 95943|1066.3008|
|  6|2000-01-12|24168.5|24452.8|23457.6|23670.9|22368.947| 61899| 995.2012|
|  7|2000-01-13|23670.9|24132.9|23102.2|23244.4|21965.906| 57538|1030.7012|
|  8|2000-01-14|23457.6|24168.5|22746.8|23244.4|21965.906| 84267|1421.6992|
|  9|2000-01-17|22533.6|23457.6|22533.6|23457.6|22167.377| 67807|    924.0|
| 10|2000-01